In [1]:
!pip install torch torchvision

In [10]:
import sys
sys.path.append('./conv')
sys.path.append('./lsh')
import alsh_conv_2d as Conv
from multi_hash_srp import MultiHash_SRP

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import time

In [12]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 7 * 7, 64)
        self.relu4 = nn.ReLU()
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        print(x.shape)
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)

        x = self.conv3(x)
        x = self.relu3(x)

        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu4(x)
        x = self.fc2(x)

        return x

In [13]:
class ALSHCNN(nn.Module):
  def __init__(self):
    super(ALSHCNN, self).__init__()
    self.conv1 = Conv.ALSHConv2d(1, 32, 3, 1, 1, 1, True, MultiHash_SRP, {}, 5, 8, 2**5, 16)
    self.relu1 = nn.ReLU()
    self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)

    #self.conv2 = Conv.ALSHConv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
    self.conv2 = Conv.ALSHConv2d(32, 64, 3, 1, 1, 1, True, MultiHash_SRP, {}, 5, 8, 2**5, 16)
    self.relu2 = nn.ReLU()
    self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)

    self.conv3 = Conv.ALSHConv2d(64, 64, 3, 1, 1, 1, True, MultiHash_SRP, {}, 5, 8, 2**5, 16)
    #self.conv3 = Conv.ALSHConv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
    self.relu3 = nn.ReLU()

    self.flatten = nn.Flatten()
    self.fc1 = nn.Linear(64 * 7 * 7, 64)
    self.relu4 = nn.ReLU()
    self.fc2 = nn.Linear(64, 10)

  def forward(self, x):
        print(x.shape)


        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)

        x = self.conv3(x)
        x = self.relu3(x)

        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu4(x)
        x = self.fc2(x)

        return x

In [14]:
def set_ALSH_mode(m):
    if isinstance(m, Conv.ALSHConv2d):
        m.ALSH_mode()

In [15]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import time

# Step 1: Load and preprocess the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

# Step 2: Define the neural network model (AlexNet)
model = models.alexnet(pretrained=True)  # Assuming CIFAR-10 has 10 classes
ftr_weight = model.features[-1].weight
r = Conv.ALSHConv2d(64, 256, 3, 1, 1, 1, True, MultiHash_SRP, {}, 5, 8, 2**5)
r.weight = ftr_weight
model.features[-1] = r

ftr_weight = model.features[-2].weight
r = Conv.ALSHConv2d(64, 256, 3, 1, 1, 1, True, MultiHash_SRP, {}, 5, 8, 2**5)
r.weight = ftr_weight
model.features[-2] = r

model.apply(set_ALSH_mode)
criterion = nn.CrossEntropyLoss()

# Step 4: Choose an optimization algorithm
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Step 5: Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 5  # You can adjust this based on your requirements

start_time = time.time()
for epoch in range(num_epochs):
    running_loss = 0.0

    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 100:.3f}')
            running_loss = 0.0

end_time = time.time()
elapsed_time = end_time - start_time
print(f'Training time: {elapsed_time:.2f} seconds')
print('Finished Training')

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1002)>

In [42]:
depth = 0

In [41]:
def replace_next_conv(model, current):
    while not isinstance(model.features[current], nn.Conv2d):
        current -= 1
    if isinstance(model.features[current], nn.Conv2d):
        print('REPLACED REGULAR Conv2d WITH ALSHConv2d')
        model.features[current] = Conv.ALSHConv2d.build(
            model.features[current], MultiHash_SRP, {}, 5, 3, 2**5, 64)
    return current-1

In [8]:
r = Conv.ALSHConv2d(64, 256, 3, 1, 1, 1, True, MultiHash_SRP, {}, 5, 8, 2**5, 64)
model.features[-1] = r

In [9]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

Files already downloaded and verified


In [14]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 117688868.63it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 31961924.49it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 25176619.15it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 14142931.53it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
# Load and preprocess the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize the model, loss function, and optimizer
# model = ALSHCNN()
# depth = 3
# current = 0
# for d in range(depth):
#         replace_next_conv(model.features[current])

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
epochs = 5

start_time = time.time()
for epoch in range(epochs):
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

end_time = time.time()
elapsed_time = end_time - start_time
print(f'Training time: {elapsed_time:.2f} seconds')
# Test the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')

In [10]:
class Model(nn.Module):
    def __init__(self, num_classes=10):
        super(Model, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1),
            nn.Hardshrink(lambd=0.3),
            #nn.Softshrink(),
            #nn.ReLU(),
            #nn.Tanh(),
            #nn.ELU(),
            nn.Conv2d(32, 32, kernel_size=3, stride=1),
            nn.Hardshrink(lambd=0.3),
            #nn.Softshrink(),
            #nn.ReLU(),
            #nn.Tanh(),
            #nn.ELU(),
            nn.Conv2d(32, 32, kernel_size=3, stride=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2))
        self.classifier = nn.Sequential(nn.Dropout(),
                                        nn.Linear(3872, num_classes),
                                        nn.Softmax(dim=1))

    def forward(self, x):
        print(x.shape)
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.classifier = nn.Sequential(
            nn.Linear(32 * 56*56, 128),  # Adjust input size based on your input dimensions
            nn.ReLU(),
            nn.Linear(128, 16)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.reshape(16, 32*56*56)  # Adjust view size based on your input dimensions
        x = self.classifier(x)
        return x

# Create an instance of the model
model = SimpleCNN(num_classes=10)

# Print the model architecture
print(model)


SimpleCNN(
  (features): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=100352, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=16, bias=True)
  )
)


In [12]:
def fix(m):
    if isinstance(m, Conv.ALSHConv2d):
        m.fix()
        m.cuda()

In [13]:
def adjust_learning_rate(optimizer, epoch):
    lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [33]:
import time
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        data_time.update(time.time() - end)

        input = input.cuda()
        target = target.cuda()
        output = model(input)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        batch_time.update(time.time() - end)
        end = time.time()

        if i % 1000 == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Data {data_time.val:.3f} ({data_time.val:.3f})\t'.format(
                      epoch,
                      i,
                      len(train_loader),
                      data_time=data_time,
                      loss=loss.item(),
                      ))
            print("Loss:" + str(loss.item()))

In [15]:
train_sampler = None
train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=16,
                                               shuffle=(train_sampler is None),
                                               num_workers=2,
                                               pin_memory=True,
                                               sampler=train_sampler)

In [40]:
model = SimpleCNN()
model = model.cuda()

for i in range(depth):
  replace_next_conv(model, depth - i - 1)

In [43]:
import time

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), 0.001)

current_depth = len(model.features) - 1
flag = True
start_time = time.time()
#replace_next_conv(model, current_depth)
for epoch in range(5):
    train(train_loader, model, criterion, optimizer, epoch)

end_time = time.time()
print(end_time - start_time)

Epoch: [0][0/3125]	Data 0.078 (0.078)	
Loss:2.789382219314575
Epoch: [0][1000/3125]	Data 0.020 (0.020)	
Loss:1.0489041805267334
Epoch: [0][2000/3125]	Data 0.019 (0.019)	
Loss:1.5077332258224487
Epoch: [0][3000/3125]	Data 0.021 (0.021)	
Loss:1.534446120262146
Epoch: [1][0/3125]	Data 0.072 (0.072)	
Loss:1.3543429374694824
Epoch: [1][1000/3125]	Data 0.020 (0.020)	
Loss:1.2730414867401123
Epoch: [1][2000/3125]	Data 0.012 (0.012)	
Loss:1.3833974599838257
Epoch: [1][3000/3125]	Data 0.000 (0.000)	
Loss:0.9389098882675171
Epoch: [2][0/3125]	Data 0.073 (0.073)	
Loss:0.7708405256271362
Epoch: [2][1000/3125]	Data 0.008 (0.008)	
Loss:1.1802189350128174
Epoch: [2][2000/3125]	Data 0.000 (0.000)	
Loss:1.2445745468139648
Epoch: [2][3000/3125]	Data 0.000 (0.000)	
Loss:1.2946547269821167
Epoch: [3][0/3125]	Data 0.076 (0.076)	
Loss:1.0959255695343018
Epoch: [3][1000/3125]	Data 0.000 (0.000)	
Loss:1.117733359336853
Epoch: [3][2000/3125]	Data 0.017 (0.017)	
Loss:1.2720227241516113
Epoch: [3][3000/3125]	Dat

In [17]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count